In [42]:
import numpy as np
img = np.array([np.random.randint(10, size = 2) for i in np.arange(0,7)])
cor = np.array([1,3,5,2,5,3,6])
res = np.array([1,2,5,1,8,3,6])
t = cor != res
indices = [i for i, x in enumerate(t) if x]
list(img[indices])


[array([4, 0]), array([8, 7]), array([3, 6])]

In [1]:
from matplotlib import pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F

from torchvision import datasets, transforms

import numpy as np

import timeit
from Network import Net
from Network import MnistModel

import sys
import os

In [3]:
print("Python {}.{} is used lol".format(*sys.version_info))

Python 3.6 is used lol


In [4]:
model = MnistModel()
model

MnistModel (
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear (3136 -> 50)
  (fc2): Linear (50 -> 10)
)

In [6]:
HOME_DIR = os.path.expanduser('~')
MNIST_DIR = HOME_DIR + '/MNIST/'

batch_size = 50
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(MNIST_DIR, train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [7]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(MNIST_DIR, train=False, transform=transforms.ToTensor()),
    batch_size=1000)

In [8]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [29]:
model.train()
train_loss = []
train_accu = []
i = 0
for epoch in range(15):
    for data, target in train_loader:
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.data[0])
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/batch_size*100
        train_accu.append(accuracy)
        if i % 1000 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.data[0], accuracy))
        i += 1

Train Step: 0	Loss: 2.301	Accuracy: 10.000
Train Step: 1000	Loss: 0.218	Accuracy: 96.000
Train Step: 2000	Loss: 0.051	Accuracy: 96.000
Train Step: 3000	Loss: 0.048	Accuracy: 98.000
Train Step: 4000	Loss: 0.058	Accuracy: 98.000
Train Step: 5000	Loss: 0.002	Accuracy: 100.000
Train Step: 6000	Loss: 0.003	Accuracy: 100.000
Train Step: 7000	Loss: 0.050	Accuracy: 98.000
Train Step: 8000	Loss: 0.023	Accuracy: 98.000
Train Step: 9000	Loss: 0.015	Accuracy: 100.000
Train Step: 10000	Loss: 0.011	Accuracy: 100.000
Train Step: 11000	Loss: 0.074	Accuracy: 98.000
Train Step: 12000	Loss: 0.004	Accuracy: 100.000
Train Step: 13000	Loss: 0.003	Accuracy: 100.000
Train Step: 14000	Loss: 0.003	Accuracy: 100.000
Train Step: 15000	Loss: 0.005	Accuracy: 100.000
Train Step: 16000	Loss: 0.003	Accuracy: 100.000
Train Step: 17000	Loss: 0.051	Accuracy: 98.000


In [9]:
plt.plot(np.arange(len(train_loss)), train_loss)

NameError: name 'train_loss' is not defined

In [ ]:
plt.plot(np.arange(len(train_accu)), train_accu)

In [ ]:
model.eval()
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('\nTest set: Accuracy: {:.2f}%'.format(100. * correct / len(test_loader.dataset)))